In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import copy

# Paths
BASE_DIR = "/kaggle/input/soil-classification/soil_classification-2025"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")
LABELS_CSV = os.path.join(BASE_DIR, "train_labels.csv")
TEST_IDS_CSV = os.path.join(BASE_DIR, "test_ids.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
train_df = pd.read_csv(LABELS_CSV)
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['soil_type'])
num_classes = len(label_encoder.classes_)

In [ ]:
class SoilDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.loc[idx, 'image_id']
        img_path = os.path.join(self.image_dir, img_id)
        image = Image.open(img_path).convert("RGB")
        label = self.df.loc[idx, 'label']
        if self.transform:
            image = self.transform(image)
        return image, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_df = train_df.reset_index(drop=True)

fold_models = []
criterion = nn.CrossEntropyLoss()

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    print(f"\n--- Fold {fold+1} ---")

    train_data = train_df.iloc[train_idx]
    val_data = train_df.iloc[val_idx]

    train_dataset = SoilDataset(train_data, TRAIN_DIR, transform=train_transform)
    val_dataset = SoilDataset(val_data, TRAIN_DIR, transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        print(f"Epoch {epoch+1}: Train Acc = {train_acc:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        print(f"Val Acc = {val_acc:.4f}")
        scheduler.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    fold_models.append(copy.deepcopy(model))


In [ ]:
tta_transforms = [
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=1.0),  # Horizontal flip
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation(15),  # Rotate ±15 degrees
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
]


In [ ]:
class TTASoilDataset(Dataset):
    def __init__(self, image_ids, image_dir, transforms_list):
        self.image_ids = image_ids
        self.image_dir = image_dir
        self.transforms_list = transforms_list

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, img_id)
        image = Image.open(img_path).convert("RGB")

        # Apply each TTA transform and stack the results
        images = [tf(image) for tf in self.transforms_list]
        images = torch.stack(images)  # Shape: [num_tta, C, H, W]

        return img_id, images

In [ ]:
# Load test image IDs (REQUIRED before preparing test dataset)
test_ids_df = pd.read_csv(TEST_IDS_CSV)
image_ids = test_ids_df["image_id"].tolist()

# Now prepare test dataset and loader
test_dataset = TTASoilDataset(image_ids, TEST_DIR, tta_transforms)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
model_preds = {img_id: [] for img_id in image_ids}

for fold, model in enumerate(fold_models):
    model.eval()
    model.to(device)
    print(f"Running TTA predictions for Fold {fold+1} ...")

    with torch.no_grad():
        for img_id, images in tqdm(test_loader):
            # images shape: [batch=1, num_tta, C, H, W]
            images = images.squeeze(0).to(device)  # shape: [num_tta, C, H, W]

            # Predict for each TTA image and average
            outputs = model(images)  # [num_tta, num_classes]
            probs = torch.softmax(outputs, dim=1)
            mean_prob = probs.mean(dim=0).cpu().numpy()  # average across TTA

            model_preds[img_id[0]].append(mean_prob)

In [ ]:
final_preds = []
for img_id in image_ids:
    # Stack fold predictions and average
    fold_probs = np.stack(model_preds[img_id], axis=0)
    avg_probs = np.mean(fold_probs, axis=0)
    pred_label = label_encoder.classes_[np.argmax(avg_probs)]
    final_preds.append(pred_label)

In [ ]:
submission = pd.DataFrame({
    "image_id": image_ids,
    "soil_type": final_preds
})

submission.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")
